In [1]:
# cell 1 
# Getting Packages
from __future__ import print_function
import Nio, Ngl
import os, math, sys
import numpy as np
import xarray as xr
import mpl_scatter_density
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
# cell 2
# Locating Files
fn = "CCTM_ACONC_v53_intel17.2_soas_2016_cb6_20130715.nc"
fn_2 = "CCTM_ACONC_v53_intel17.2_soas_2016_cb6_20130601.nc"
fn_MCIP = "GRIDCRO2D_20130715.nc4"
fn_MCIP_2 = "GRIDCRO2D_20130601.nc4"

# Reading Files
ACON_file = Nio.open_file(fn, mode = "r")
ACON_file_MCIP = Nio.open_file(fn_MCIP, mode = "r")
ACON_file_2 = Nio.open_file(fn_2, mode = "r")
ACON_file_MCIP_2 = Nio.open_file(fn_MCIP_2, mode = "r")

In [3]:
# cell 3 
# obtaining data for from first file
ds = xr.open_dataset(fn)
ds_MCIP = xr.open_dataset(fn_MCIP)

var_O3 = ds['O3'][4:,0,:,:]
var_NO = ds['NO'][4:,0,:,:]
var_NO2 = ds['NO2'][4:,0,:,:]
var_CO = ds['CO'][4:,0,:,:]
var_SO2 = ds['SO2'][4:,0,:,:]
lat2d = ds_MCIP['LAT'][0,0,:,:]   
lon2d = ds_MCIP['LON'][0,0,:,:]  

# checking shape
print('var_O3:{}'.format(var_O3.shape))
print('var_NO:{}'.format(var_NO.shape))
print('var_NO2:{}'.format(var_NO2.shape))
print('var_CO:{}'.format(var_CO.shape))
print('var_SO2:{}'.format(var_SO2.shape))
print('lat2d.shape:{}'.format(lat2d.shape))
print('lon2d.shape:{}'.format(lon2d.shape))

var_O3:(20, 299, 459)
var_NO:(20, 299, 459)
var_NO2:(20, 299, 459)
var_CO:(20, 299, 459)
var_SO2:(20, 299, 459)
lat2d.shape:(299, 459)
lon2d.shape:(299, 459)


In [4]:
# cell 4 
# obtaining data from 2nd file 
ds_2 = xr.open_dataset(fn_2)
ds_MCIP_2 = xr.open_dataset(fn_MCIP_2)

var2_O3 = ds_2['O3'][:4,0,:,:]
var2_NO = ds_2['NO'][:4,0,:,:]
var2_NO2 = ds_2['NO2'][:4,0,:,:]
var2_CO = ds_2['CO'][:4,0,:,:]
var2_SO2 = ds_2['SO2'][:4,0,:,:]
lat2d_2 = ds_MCIP_2['LAT'][0,0,:,:]   
lon2d_2 = ds_MCIP_2['LON'][0,0,:,:]  

#checking shape
print('var2_O3:{}'.format(var2_O3.shape))
print('var2_NO:{}'.format(var2_NO.shape))
print('var2_NO2:{}'.format(var2_NO2.shape))
print('var2_CO:{}'.format(var2_CO.shape))
print('var2_SO2:{}'.format(var2_SO2.shape))
print('lat2d_2.shape:{}'.format(lat2d_2.shape))
print('lon2d_2.shape:{}'.format(lon2d_2.shape))

var2_O3:(4, 299, 459)
var2_NO:(4, 299, 459)
var2_NO2:(4, 299, 459)
var2_CO:(4, 299, 459)
var2_SO2:(4, 299, 459)
lat2d_2.shape:(299, 459)
lon2d_2.shape:(299, 459)


In [5]:
# cell 5
# total grid cells (for referance) 
299*459

137241

In [6]:
# cell 6
# defining new variables containing values from obtained data

# From First File
O3_Values_D1 = var_O3.values
NO_Values_D1 = var_NO.values
NO2_Values_D1 = var_NO2.values
CO_Values_D1 = var_CO.values
SO2_Values_D1 = var_SO2.values
lat2d_Values = lat2d.values
lon2d_Values = lon2d.values

# From Second File
O3_Values_D2 = var2_O3.values
NO_Values_D2 = var2_NO.values
NO2_Values_D2 = var2_NO2.values
CO_Values_D2 = var2_CO.values
SO2_Values_D2 = var2_SO2.values
lat2d_2_Values = lat2d_2.values
lon2d_2_Values = lon2d_2.values

In [7]:
# cell 7
# combining each array to make concentration values for 1 day
O3_CV = np.concatenate((O3_Values_D1, O3_Values_D2))
NO_CV = np.concatenate((NO_Values_D1, NO_Values_D2))
NO2_CV = np.concatenate((NO2_Values_D1, NO2_Values_D2))
CO_CV = np.concatenate((CO_Values_D1, CO_Values_D2))
SO2_CV = np.concatenate((SO2_Values_D1, SO2_Values_D2))

# checking shape of new variable
print ('O3_CV.shape:{}'.format(O3_CV.shape))
print ('NO_CV.shape:{}'.format(NO_CV.shape))
print ('NO2_CV.shape:{}'.format(NO2_CV.shape))
print ('CO_CV.shape:{}'.format(CO_CV.shape))
print ('SO2_CV.shape:{}'.format(SO2_CV.shape))
print ('O3_Values_D1.shape:{}'.format(O3_Values_D1.shape))
print ('O3_Values_D2.shape:{}'.format(O3_Values_D2.shape))

O3_CV.shape:(24, 299, 459)
NO_CV.shape:(24, 299, 459)
NO2_CV.shape:(24, 299, 459)
CO_CV.shape:(24, 299, 459)
SO2_CV.shape:(24, 299, 459)
O3_Values_D1.shape:(20, 299, 459)
O3_Values_D2.shape:(4, 299, 459)


In [8]:
# cell 8
# Checing that lat and lon arrays are the same from both files
print((lat2d_Values == lat2d_2_Values).all())
print((lon2d_Values == lon2d_2_Values).all())

True
True


In [9]:
# cell 9
# putting data into a data array

CMAQ_Chem_da = xr.Dataset(
    data_vars=dict(
        O3 = (["x", "y", "z"], O3_CV),
        NO = (["x", "y", "z"], NO_CV),
        NO2 = (["x", "y", "z"], NO2_CV),
        CO = (["x", "y", "z"], CO_CV),
        SO2 = (["x", "y", "z"], SO2_CV)
    ),
    coords=dict(
        lon=(["y", "z"], lon2d_Values),
        lat=(["y", "z"], lat2d_Values),
        ),
    attrs=dict(
        description="Ozone, Nitrogen Monxide, Nitrogen Dioxide, Carbon Monoxide, Sulfur Dioxide Concentrations",
        units="Parts per Million (ppm)",
    ),
)
CMAQ_Chem_da

<xarray.Dataset>
Dimensions:  (x: 24, y: 299, z: 459)
Coordinates:
    lon      (y, z) float32 -121.0 -120.9 -120.8 -120.7 ... -54.87 -54.72 -54.57
    lat      (y, z) float32 21.62 21.65 21.68 21.7 ... 50.5 50.45 50.4 50.36
Dimensions without coordinates: x, y, z
Data variables:
    O3       (x, y, z) float32 0.0197 0.01981 0.01997 ... 0.03161 0.03139
    NO       (x, y, z) float32 1.262e-11 1.342e-11 ... 8.358e-13 1.053e-12
    NO2      (x, y, z) float32 1.062e-05 1.092e-05 ... 6.164e-05 6.792e-05
    CO       (x, y, z) float32 0.06608 0.06598 0.06589 ... 0.09443 0.09378
    SO2      (x, y, z) float32 1.008e-06 8.821e-07 ... 2.817e-08 5.196e-08
Attributes:
    description:  Ozone, Nitrogen Monxide, Nitrogen Dioxide, Carbon Monoxide,...
    units:        Parts per Million (ppm)

In [10]:
# cell 10 
# extracting desired coordinates
County_CMAQ_Chem = xr.where(CMAQ_Chem_da.lat > 34.297882  , CMAQ_Chem_da, np.nan)
County_CMAQ_Chem = xr.where(County_CMAQ_Chem.lat < 34.95478058 , County_CMAQ_Chem , np.nan)
County_CMAQ_Chem = xr.where(County_CMAQ_Chem.lon < -78.80546570 , County_CMAQ_Chem , np.nan)
County_CMAQ_Chem = xr.where(County_CMAQ_Chem.lon > -79.46142578 , County_CMAQ_Chem , np.nan)
County_CMAQ_Chem

<xarray.Dataset>
Dimensions:  (y: 299, z: 459, x: 24)
Coordinates:
    lon      (y, z) float32 -121.0 -120.9 -120.8 -120.7 ... -54.87 -54.72 -54.57
    lat      (y, z) float32 21.62 21.65 21.68 21.7 ... 50.5 50.45 50.4 50.36
Dimensions without coordinates: y, z, x
Data variables:
    O3       (y, z, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    NO       (y, z, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    NO2      (y, z, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    CO       (y, z, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    SO2      (y, z, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan

In [11]:
# cell 11
# converting to a data array
County_CMAQ_ChemArr = County_CMAQ_Chem.to_array()
County_CMAQ_ChemArr

<xarray.DataArray (variable: 5, y: 299, z: 459, x: 24)>
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
...
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]], dtype=float32)
Coordinates:
    lon       (y, z) float32 -121.0 -120.9 -120.8 ... -54.87 -54.72 -54.57
    lat       (y, z) float32 21.62 21.65 21.68 21.7 ... 50.5 50.45 50.4 50.36
  * variable  (variable) object 'O3' 'NO' 'NO2' 'CO' 'SO2'
Dimensions without coordinates: y, z, x

In [12]:
# cell 12
# checking array 
County_CMAQ_ChemArr[0,105:112,346:352,:]

<xarray.DataArray (y: 7, z: 6, x: 24)>
array([[[0.02190279, 0.0193124 , 0.01719732, ..., 0.02194984,
         0.02099405, 0.01936545],
        [0.0215089 , 0.01909408, 0.01708741, ..., 0.0221952 ,
         0.02124204, 0.01938316],
        [0.02046691, 0.01813515, 0.01627668, ..., 0.02186815,
         0.02061467, 0.01884888],
        [0.01996751, 0.01764398, 0.01578245, ..., 0.02130179,
         0.01953008, 0.01782491],
        [0.0189996 , 0.01667057, 0.0148631 , ..., 0.02092302,
         0.01885156, 0.01718939],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[0.02236195, 0.0196434 , 0.01736388, ..., 0.02214072,
         0.02130949, 0.01981994],
        [0.02145355, 0.01891853, 0.016837  , ..., 0.02209491,
         0.0212778 , 0.01972195],
        [0.02105248, 0.01865944, 0.01664983, ..., 0.02186948,
         0.02080242, 0.01920375],
        [0.0200297 , 0.01769139, 0.01583093, ..., 0.02147816,
...
         0.02060909, 0.01985597],
        [0.01945928, 0.01717748, 0.01433915, ..., 0.02182914,
         0.02112257, 0.01968651],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
    lon       (y, z) float32 -79.38 -79.25 -79.13 -79.0 ... -78.84 -78.71 -78.58
    lat       (y, z) float32 34.42 34.4 34.38 34.35 ... 35.01 34.99 34.97 34.95
    variable  <U2 'O3'
Dimensions without coordinates: y, z, x

In [13]:
# cell 13
## we will use each count variable to count our real numbers for each hour
count_T0 = 0 
count_T1 = 0
count_T2 = 0
count_T3 = 0
count_T4 = 0
count_T5 = 0
count_T6 = 0
count_T7 = 0
count_T8 = 0
count_T9 = 0
count_T10 = 0
count_T11 = 0
count_T12 = 0
count_T13 = 0
count_T14 = 0
count_T15 = 0
count_T16 = 0
count_T17 = 0
count_T18 = 0
count_T19 = 0
count_T20 = 0
count_T21 = 0
count_T22 = 0
count_T23 = 0

col = 299
row = 459
time = 24
for i in range(col):
    for j in range(row):
        for k in range(time):
            if k == 0:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                    count_T0 = count_T0
                else:
                    count_T0 = count_T0 + 1
            if k == 1:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                    count_T1 = count_T1
                else:
                    count_T1 = count_T1 + 1
            if k == 2:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                    count_T2 = count_T2
                else:
                    count_T2 = count_T2 + 1
            if k == 3:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                    count_T3 = count_T3
                else:
                    count_T3 = count_T3 + 1
            if k == 4:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                    count_T4 = count_T4
                else:
                    count_T4 = count_T4 + 1
            if k == 5:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                    count_T5 = count_T5
                else:
                    count_T5 = count_T5 + 1
            if k == 6:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T6 = count_T6
                else:
                     count_T6 = count_T6 + 1
            if k == 7:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T7 = count_T7
                else:
                     count_T7 = count_T7 + 1
            if k == 8:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T8 = count_T8
                else:
                     count_T8 = count_T8 + 1
            if k == 9:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T9 = count_T9
                else:
                     count_T9 = count_T9 + 1
            if k == 10:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T10 = count_T10
                else:
                     count_T10 = count_T10 + 1
            if k == 11:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T11 = count_T11
                else:
                     count_T11 = count_T11 + 1
            if k == 12:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T12 = count_T12
                else:
                     count_T12 = count_T12 + 1
            if k == 13:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T13 = count_T13
                else:
                     count_T13 = count_T13 + 1
            if k == 14:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T14 = count_T14
                else:
                     count_T14 = count_T14 + 1
            if k == 15:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T15 = count_T15
                else:
                     count_T15 = count_T15 + 1
            if k == 16:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T16 = count_T16
                else:
                     count_T16 = count_T16 + 1
            if k == 17:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T17 = count_T17
                else:
                     count_T17 = count_T17 + 1
            if k == 18:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T18 = count_T18
                else:
                     count_T18 = count_T18 + 1
            if k == 19:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T19 = count_T19
                else:
                     count_T19 = count_T19 + 1
            if k == 20:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T20 = count_T20
                else:
                     count_T20 = count_T20 + 1
            if k == 21:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T21 = count_T21
                else:
                     count_T21 = count_T21 + 1
            if k == 22:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T22 = count_T22
                else:
                     count_T22 = count_T22 + 1
            if k == 23:
                if np.isnan(County_CMAQ_ChemArr[0,i,j,k]):
                     count_T23 = count_T23
                else:
                     count_T23 = count_T23 + 1
            
                    
            
                
 ## if there are zero, check the dimensions of the box for lat and lon            
print(count_T0)
print(count_T1)
print(count_T2)
print(count_T3)
print(count_T4)
print(count_T5)
print(count_T6)
print(count_T7)
print(count_T8)
print(count_T9)
print(count_T10)
print(count_T11)
print(count_T12)
print(count_T13)
print(count_T14)
print(count_T15)
print(count_T16)
print(count_T17)
print(count_T18)
print(count_T19)
print(count_T20)
print(count_T21)
print(count_T22)
print(count_T23)

30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30


In [14]:
# cell 14
# creating Lat and Lon arrays
#Latitude
#D0
output0_Lat = xr.DataArray(np.arange(count_T0, dtype=np.double))
output0_Lat = xr.zeros_like(output0_Lat)
#D1
output1_Lat = xr.DataArray(np.arange(count_T1, dtype=np.double))
output1_Lat = xr.zeros_like(output1_Lat)
#D2
output2_Lat = xr.DataArray(np.arange(count_T2, dtype=np.double))
output2_Lat = xr.zeros_like(output2_Lat)
#D3
output3_Lat = xr.DataArray(np.arange(count_T3, dtype=np.double))
output3_Lat = xr.zeros_like(output3_Lat)
#D4
output4_Lat = xr.DataArray(np.arange(count_T4, dtype=np.double))
output4_Lat = xr.zeros_like(output4_Lat)
#D5
output5_Lat = xr.DataArray(np.arange(count_T5, dtype=np.double))
output5_Lat = xr.zeros_like(output5_Lat)
#D6
output6_Lat = xr.DataArray(np.arange(count_T6, dtype=np.double))
output6_Lat = xr.zeros_like(output6_Lat)
#D7
output7_Lat = xr.DataArray(np.arange(count_T7, dtype=np.double))
output7_Lat = xr.zeros_like(output7_Lat)
#D8
output8_Lat = xr.DataArray(np.arange(count_T8, dtype=np.double))
output8_Lat = xr.zeros_like(output8_Lat)
#D9
output9_Lat = xr.DataArray(np.arange(count_T9, dtype=np.double))
output9_Lat = xr.zeros_like(output9_Lat)
#D10
output10_Lat = xr.DataArray(np.arange(count_T10, dtype=np.double))
output10_Lat = xr.zeros_like(output10_Lat)
#D11
output11_Lat = xr.DataArray(np.arange(count_T11, dtype=np.double))
output11_Lat = xr.zeros_like(output11_Lat)
#D12
output12_Lat = xr.DataArray(np.arange(count_T12, dtype=np.double))
output12_Lat = xr.zeros_like(output12_Lat)
#D13
output13_Lat = xr.DataArray(np.arange(count_T13, dtype=np.double))
output13_Lat = xr.zeros_like(output13_Lat)
#D14
output14_Lat = xr.DataArray(np.arange(count_T14, dtype=np.double))
output14_Lat = xr.zeros_like(output14_Lat)
#D15
output15_Lat = xr.DataArray(np.arange(count_T15, dtype=np.double))
output15_Lat = xr.zeros_like(output15_Lat)
#D16
output16_Lat = xr.DataArray(np.arange(count_T16, dtype=np.double))
output16_Lat = xr.zeros_like(output16_Lat)
#D17
output17_Lat = xr.DataArray(np.arange(count_T17, dtype=np.double))
output17_Lat = xr.zeros_like(output17_Lat)
#D18
output18_Lat = xr.DataArray(np.arange(count_T18, dtype=np.double))
output18_Lat = xr.zeros_like(output18_Lat)
#D19
output19_Lat = xr.DataArray(np.arange(count_T19, dtype=np.double))
output19_Lat = xr.zeros_like(output19_Lat)
#D20
output20_Lat = xr.DataArray(np.arange(count_T20, dtype=np.double))
output20_Lat = xr.zeros_like(output20_Lat)
#D21
output21_Lat = xr.DataArray(np.arange(count_T21, dtype=np.double))
output21_Lat = xr.zeros_like(output21_Lat)
#D22
output22_Lat = xr.DataArray(np.arange(count_T22, dtype=np.double))
output22_Lat = xr.zeros_like(output22_Lat)
#D23
output23_Lat = xr.DataArray(np.arange(count_T23, dtype=np.double))
output23_Lat = xr.zeros_like(output23_Lat)

# Longitude
#D0
output0_Lon = xr.DataArray(np.arange(count_T0, dtype=np.double))
output0_Lon = xr.DataArray(xr.zeros_like(output0_Lon))
#D1
output1_Lon = xr.DataArray(np.arange(count_T1, dtype=np.double))
output1_Lon = xr.DataArray(xr.zeros_like(output1_Lon))
#D2
output2_Lon = xr.DataArray(np.arange(count_T2, dtype=np.double))
output2_Lon = xr.DataArray(xr.zeros_like(output2_Lon))
#D3
output3_Lon = xr.DataArray(np.arange(count_T3, dtype=np.double))
output3_Lon = xr.DataArray(xr.zeros_like(output3_Lon))
#D4
output4_Lon = xr.DataArray(np.arange(count_T4, dtype=np.double))
output4_Lon = xr.DataArray(xr.zeros_like(output4_Lon))
#D5
output5_Lon = xr.DataArray(np.arange(count_T5, dtype=np.double))
output5_Lon = xr.DataArray(xr.zeros_like(output5_Lon))
#D6
output6_Lon = xr.DataArray(np.arange(count_T6, dtype=np.double))
output6_Lon = xr.DataArray(xr.zeros_like(output6_Lon))
#D7
output7_Lon = xr.DataArray(np.arange(count_T7, dtype=np.double))
output7_Lon = xr.DataArray(xr.zeros_like(output7_Lon))
#D8
output8_Lon = xr.DataArray(np.arange(count_T8, dtype=np.double))
output8_Lon = xr.DataArray(xr.zeros_like(output8_Lon))
#D9
output9_Lon = xr.DataArray(np.arange(count_T9, dtype=np.double))
output9_Lon = xr.DataArray(xr.zeros_like(output9_Lon))
#D10
output10_Lon = xr.DataArray(np.arange(count_T10, dtype=np.double))
output10_Lon = xr.DataArray(xr.zeros_like(output10_Lon))
#D11
output11_Lon = xr.DataArray(np.arange(count_T11, dtype=np.double))
output11_Lon = xr.DataArray(xr.zeros_like(output11_Lon))
#D12
output12_Lon = xr.DataArray(np.arange(count_T12, dtype=np.double))
output12_Lon = xr.DataArray(xr.zeros_like(output12_Lon))
#D13
output13_Lon = xr.DataArray(np.arange(count_T13, dtype=np.double))
output13_Lon = xr.DataArray(xr.zeros_like(output13_Lon))
#D14
output14_Lon = xr.DataArray(np.arange(count_T14, dtype=np.double))
output14_Lon = xr.DataArray(xr.zeros_like(output14_Lon))
#D15
output15_Lon = xr.DataArray(np.arange(count_T15, dtype=np.double))
output15_Lon = xr.DataArray(xr.zeros_like(output15_Lon))
#D16
output16_Lon = xr.DataArray(np.arange(count_T16, dtype=np.double))
output16_Lon = xr.DataArray(xr.zeros_like(output16_Lon))
#D17
output17_Lon = xr.DataArray(np.arange(count_T17, dtype=np.double))
output17_Lon = xr.DataArray(xr.zeros_like(output17_Lon))
#D18
output18_Lon = xr.DataArray(np.arange(count_T18, dtype=np.double))
output18_Lon = xr.DataArray(xr.zeros_like(output18_Lon))
#D19
output19_Lon = xr.DataArray(np.arange(count_T19, dtype=np.double))
output19_Lon = xr.DataArray(xr.zeros_like(output19_Lon))
#D20
output20_Lon = xr.DataArray(np.arange(count_T20, dtype=np.double))
output20_Lon = xr.DataArray(xr.zeros_like(output20_Lon))
#D21
output21_Lon = xr.DataArray(np.arange(count_T21, dtype=np.double))
output21_Lon = xr.DataArray(xr.zeros_like(output21_Lon))
#D22
output22_Lon = xr.DataArray(np.arange(count_T22, dtype=np.double))
output22_Lon = xr.DataArray(xr.zeros_like(output22_Lon))
#D23
output23_Lon = xr.DataArray(np.arange(count_T23, dtype=np.double))
output23_Lon = xr.DataArray(xr.zeros_like(output23_Lon))

In [15]:
# cell 15
# creating arrays for each hour
#D0
output0_O3 = xr.DataArray(np.arange(count_T0, dtype=np.double))
output0_O3 = xr.DataArray(xr.zeros_like(output0_O3))
#D1
output1_O3 = xr.DataArray(np.arange(count_T1, dtype=np.double))
output1_O3 = xr.DataArray(xr.zeros_like(output1_O3))
#D2
output2_O3 = xr.DataArray(np.arange(count_T2, dtype=np.double))
output2_O3 = xr.DataArray(xr.zeros_like(output2_O3))
#D3
output3_O3 = xr.DataArray(np.arange(count_T3, dtype=np.double))
output3_O3 = xr.DataArray(xr.zeros_like(output3_O3))
#D4
output4_O3 = xr.DataArray(np.arange(count_T4, dtype=np.double))
output4_O3 = xr.DataArray(xr.zeros_like(output4_O3))
#D5
output5_O3 = xr.DataArray(np.arange(count_T5, dtype=np.double))
output5_O3 = xr.DataArray(xr.zeros_like(output5_O3))
#D6
output6_O3 = xr.DataArray(np.arange(count_T6, dtype=np.double))
output6_O3 = xr.DataArray(xr.zeros_like(output6_O3))
#D7
output7_O3 = xr.DataArray(np.arange(count_T7, dtype=np.double))
output7_O3 = xr.DataArray(xr.zeros_like(output7_O3))
#D8
output8_O3 = xr.DataArray(np.arange(count_T8, dtype=np.double))
output8_O3 = xr.DataArray(xr.zeros_like(output8_O3))
#D9
output9_O3 = xr.DataArray(np.arange(count_T9, dtype=np.double))
output9_O3 = xr.DataArray(xr.zeros_like(output9_O3))
#D10
output10_O3 = xr.DataArray(np.arange(count_T10, dtype=np.double))
output10_O3 = xr.DataArray(xr.zeros_like(output10_O3))
#D11
output11_O3 = xr.DataArray(np.arange(count_T11, dtype=np.double))
output11_O3 = xr.DataArray(xr.zeros_like(output11_O3))
#D12
output12_O3 = xr.DataArray(np.arange(count_T12, dtype=np.double))
output12_O3 = xr.DataArray(xr.zeros_like(output12_O3))
#D13
output13_O3 = xr.DataArray(np.arange(count_T13, dtype=np.double))
output13_O3 = xr.DataArray(xr.zeros_like(output13_O3))
#D14
output14_O3 = xr.DataArray(np.arange(count_T14, dtype=np.double))
output14_O3 = xr.DataArray(xr.zeros_like(output14_O3))
#D15
output15_O3 = xr.DataArray(np.arange(count_T15, dtype=np.double))
output15_O3 = xr.DataArray(xr.zeros_like(output15_O3))
#D16
output16_O3 = xr.DataArray(np.arange(count_T16, dtype=np.double))
output16_O3 = xr.DataArray(xr.zeros_like(output16_O3))
#D17
output17_O3 = xr.DataArray(np.arange(count_T17, dtype=np.double))
output17_O3 = xr.DataArray(xr.zeros_like(output17_O3))
#D18
output18_O3 = xr.DataArray(np.arange(count_T18, dtype=np.double))
output18_O3 = xr.DataArray(xr.zeros_like(output18_O3))
#D19
output19_O3 = xr.DataArray(np.arange(count_T19, dtype=np.double))
output19_O3 = xr.DataArray(xr.zeros_like(output19_O3))
#D20
output20_O3 = xr.DataArray(np.arange(count_T20, dtype=np.double))
output20_O3 = xr.DataArray(xr.zeros_like(output20_O3))
#D21
output21_O3 = xr.DataArray(np.arange(count_T21, dtype=np.double))
output21_O3 = xr.DataArray(xr.zeros_like(output21_O3))
#D22
output22_O3 = xr.DataArray(np.arange(count_T22, dtype=np.double))
output22_O3 = xr.DataArray(xr.zeros_like(output22_O3))
#D23
output23_O3 = xr.DataArray(np.arange(count_T23, dtype=np.double))
output23_O3 = xr.DataArray(xr.zeros_like(output23_O3))

#D0
output0_NO = xr.DataArray(np.arange(count_T0, dtype=np.double))
output0_NO = xr.DataArray(xr.zeros_like(output0_NO))
#D1
output1_NO = xr.DataArray(np.arange(count_T1, dtype=np.double))
output1_NO = xr.DataArray(xr.zeros_like(output1_NO))
#D2
output2_NO = xr.DataArray(np.arange(count_T2, dtype=np.double))
output2_NO = xr.DataArray(xr.zeros_like(output2_NO))
#D3
output3_NO = xr.DataArray(np.arange(count_T3, dtype=np.double))
output3_NO = xr.DataArray(xr.zeros_like(output3_NO))
#D4
output4_NO = xr.DataArray(np.arange(count_T4, dtype=np.double))
output4_NO = xr.DataArray(xr.zeros_like(output4_NO))
#D5
output5_NO = xr.DataArray(np.arange(count_T5, dtype=np.double))
output5_NO = xr.DataArray(xr.zeros_like(output5_NO))
#D6
output6_NO = xr.DataArray(np.arange(count_T6, dtype=np.double))
output6_NO = xr.DataArray(xr.zeros_like(output6_NO))
#D7
output7_NO = xr.DataArray(np.arange(count_T7, dtype=np.double))
output7_NO = xr.DataArray(xr.zeros_like(output7_NO))
#D8
output8_NO = xr.DataArray(np.arange(count_T8, dtype=np.double))
output8_NO = xr.DataArray(xr.zeros_like(output8_NO))
#D9
output9_NO = xr.DataArray(np.arange(count_T9, dtype=np.double))
output9_NO = xr.DataArray(xr.zeros_like(output9_NO))
#D10
output10_NO = xr.DataArray(np.arange(count_T10, dtype=np.double))
output10_NO = xr.DataArray(xr.zeros_like(output10_NO))
#D11
output11_NO = xr.DataArray(np.arange(count_T11, dtype=np.double))
output11_NO = xr.DataArray(xr.zeros_like(output11_NO))
#D12
output12_NO = xr.DataArray(np.arange(count_T12, dtype=np.double))
output12_NO = xr.DataArray(xr.zeros_like(output12_NO))
#D13
output13_NO = xr.DataArray(np.arange(count_T13, dtype=np.double))
output13_NO = xr.DataArray(xr.zeros_like(output13_NO))
#D14
output14_NO = xr.DataArray(np.arange(count_T14, dtype=np.double))
output14_NO = xr.DataArray(xr.zeros_like(output14_NO))
#D15
output15_NO = xr.DataArray(np.arange(count_T15, dtype=np.double))
output15_NO = xr.DataArray(xr.zeros_like(output15_NO))
#D16
output16_NO = xr.DataArray(np.arange(count_T16, dtype=np.double))
output16_NO = xr.DataArray(xr.zeros_like(output16_NO))
#D17
output17_NO = xr.DataArray(np.arange(count_T17, dtype=np.double))
output17_NO = xr.DataArray(xr.zeros_like(output17_NO))
#D18
output18_NO = xr.DataArray(np.arange(count_T18, dtype=np.double))
output18_NO = xr.DataArray(xr.zeros_like(output18_NO))
#D19
output19_NO = xr.DataArray(np.arange(count_T19, dtype=np.double))
output19_NO = xr.DataArray(xr.zeros_like(output19_NO))
#D20
output20_NO = xr.DataArray(np.arange(count_T20, dtype=np.double))
output20_NO = xr.DataArray(xr.zeros_like(output20_NO))
#D21
output21_NO = xr.DataArray(np.arange(count_T21, dtype=np.double))
output21_NO = xr.DataArray(xr.zeros_like(output21_NO))
#D22
output22_NO = xr.DataArray(np.arange(count_T22, dtype=np.double))
output22_NO = xr.DataArray(xr.zeros_like(output22_NO))
#D23
output23_NO = xr.DataArray(np.arange(count_T23, dtype=np.double))
output23_NO = xr.DataArray(xr.zeros_like(output23_NO))

In [16]:
#D0
output0_NO2 = xr.DataArray(np.arange(count_T0, dtype=np.double))
output0_NO2 = xr.DataArray(xr.zeros_like(output0_NO2))
#D1
output1_NO2 = xr.DataArray(np.arange(count_T1, dtype=np.double))
output1_NO2 = xr.DataArray(xr.zeros_like(output1_NO2))
#D2
output2_NO2 = xr.DataArray(np.arange(count_T2, dtype=np.double))
output2_NO2 = xr.DataArray(xr.zeros_like(output2_NO2))
#D3
output3_NO2 = xr.DataArray(np.arange(count_T3, dtype=np.double))
output3_NO2 = xr.DataArray(xr.zeros_like(output3_NO2))
#D4
output4_NO2 = xr.DataArray(np.arange(count_T4, dtype=np.double))
output4_NO2 = xr.DataArray(xr.zeros_like(output4_NO2))
#D5
output5_NO2 = xr.DataArray(np.arange(count_T5, dtype=np.double))
output5_NO2 = xr.DataArray(xr.zeros_like(output5_NO2))
#D6
output6_NO2 = xr.DataArray(np.arange(count_T6, dtype=np.double))
output6_NO2 = xr.DataArray(xr.zeros_like(output6_NO2))
#D7
output7_NO2 = xr.DataArray(np.arange(count_T7, dtype=np.double))
output7_NO2 = xr.DataArray(xr.zeros_like(output7_NO2))
#D8
output8_NO2 = xr.DataArray(np.arange(count_T8, dtype=np.double))
output8_NO2 = xr.DataArray(xr.zeros_like(output8_NO2))
#D9
output9_NO2 = xr.DataArray(np.arange(count_T9, dtype=np.double))
output9_NO2 = xr.DataArray(xr.zeros_like(output9_NO2))
#D10
output10_NO2 = xr.DataArray(np.arange(count_T10, dtype=np.double))
output10_NO2 = xr.DataArray(xr.zeros_like(output10_NO2))
#D11
output11_NO2 = xr.DataArray(np.arange(count_T11, dtype=np.double))
output11_NO2 = xr.DataArray(xr.zeros_like(output11_NO2))
#D12
output12_NO2 = xr.DataArray(np.arange(count_T12, dtype=np.double))
output12_NO2 = xr.DataArray(xr.zeros_like(output12_NO2))
#D13
output13_NO2 = xr.DataArray(np.arange(count_T13, dtype=np.double))
output13_NO2 = xr.DataArray(xr.zeros_like(output13_NO2))
#D14
output14_NO2 = xr.DataArray(np.arange(count_T14, dtype=np.double))
output14_NO2 = xr.DataArray(xr.zeros_like(output14_NO2))
#D15
output15_NO2 = xr.DataArray(np.arange(count_T15, dtype=np.double))
output15_NO2 = xr.DataArray(xr.zeros_like(output15_NO2))
#D16
output16_NO2 = xr.DataArray(np.arange(count_T16, dtype=np.double))
output16_NO2 = xr.DataArray(xr.zeros_like(output16_NO2))
#D17
output17_NO2 = xr.DataArray(np.arange(count_T17, dtype=np.double))
output17_NO2 = xr.DataArray(xr.zeros_like(output17_NO2))
#D18
output18_NO2 = xr.DataArray(np.arange(count_T18, dtype=np.double))
output18_NO2 = xr.DataArray(xr.zeros_like(output18_NO2))
#D19
output19_NO2 = xr.DataArray(np.arange(count_T19, dtype=np.double))
output19_NO2 = xr.DataArray(xr.zeros_like(output19_NO2))
#D20
output20_NO2 = xr.DataArray(np.arange(count_T20, dtype=np.double))
output20_NO2 = xr.DataArray(xr.zeros_like(output20_NO2))
#D21
output21_NO2 = xr.DataArray(np.arange(count_T21, dtype=np.double))
output21_NO2 = xr.DataArray(xr.zeros_like(output21_NO2))
#D22
output22_NO2 = xr.DataArray(np.arange(count_T22, dtype=np.double))
output22_NO2 = xr.DataArray(xr.zeros_like(output22_NO2))
#D23
output23_NO2 = xr.DataArray(np.arange(count_T23, dtype=np.double))
output23_NO2 = xr.DataArray(xr.zeros_like(output23_NO2))

In [17]:
#D0
output0_CO = xr.DataArray(np.arange(count_T0, dtype=np.double))
output0_CO = xr.DataArray(xr.zeros_like(output0_CO))
#D1
output1_CO = xr.DataArray(np.arange(count_T1, dtype=np.double))
output1_CO = xr.DataArray(xr.zeros_like(output1_CO))
#D2
output2_CO = xr.DataArray(np.arange(count_T2, dtype=np.double))
output2_CO = xr.DataArray(xr.zeros_like(output2_CO))
#D3
output3_CO = xr.DataArray(np.arange(count_T3, dtype=np.double))
output3_CO = xr.DataArray(xr.zeros_like(output3_CO))
#D4
output4_CO = xr.DataArray(np.arange(count_T4, dtype=np.double))
output4_CO = xr.DataArray(xr.zeros_like(output4_CO))
#D5
output5_CO = xr.DataArray(np.arange(count_T5, dtype=np.double))
output5_CO = xr.DataArray(xr.zeros_like(output5_CO))
#D6
output6_CO = xr.DataArray(np.arange(count_T6, dtype=np.double))
output6_CO = xr.DataArray(xr.zeros_like(output6_CO))
#D7
output7_CO = xr.DataArray(np.arange(count_T7, dtype=np.double))
output7_CO = xr.DataArray(xr.zeros_like(output7_CO))
#D8
output8_CO = xr.DataArray(np.arange(count_T8, dtype=np.double))
output8_CO = xr.DataArray(xr.zeros_like(output8_CO))
#D9
output9_CO = xr.DataArray(np.arange(count_T9, dtype=np.double))
output9_CO = xr.DataArray(xr.zeros_like(output9_CO))
#D10
output10_CO = xr.DataArray(np.arange(count_T10, dtype=np.double))
output10_CO = xr.DataArray(xr.zeros_like(output10_CO))
#D11
output11_CO = xr.DataArray(np.arange(count_T11, dtype=np.double))
output11_CO = xr.DataArray(xr.zeros_like(output11_CO))
#D12
output12_CO = xr.DataArray(np.arange(count_T12, dtype=np.double))
output12_CO = xr.DataArray(xr.zeros_like(output12_CO))
#D13
output13_CO = xr.DataArray(np.arange(count_T13, dtype=np.double))
output13_CO = xr.DataArray(xr.zeros_like(output13_CO))
#D14
output14_CO = xr.DataArray(np.arange(count_T14, dtype=np.double))
output14_CO = xr.DataArray(xr.zeros_like(output14_CO))
#D15
output15_CO = xr.DataArray(np.arange(count_T15, dtype=np.double))
output15_CO = xr.DataArray(xr.zeros_like(output15_CO))
#D16
output16_CO = xr.DataArray(np.arange(count_T16, dtype=np.double))
output16_CO = xr.DataArray(xr.zeros_like(output16_CO))
#D17
output17_CO = xr.DataArray(np.arange(count_T17, dtype=np.double))
output17_CO = xr.DataArray(xr.zeros_like(output17_CO))
#D18
output18_CO = xr.DataArray(np.arange(count_T18, dtype=np.double))
output18_CO = xr.DataArray(xr.zeros_like(output18_CO))
#D19
output19_CO = xr.DataArray(np.arange(count_T19, dtype=np.double))
output19_CO = xr.DataArray(xr.zeros_like(output19_CO))
#D20
output20_CO = xr.DataArray(np.arange(count_T20, dtype=np.double))
output20_CO = xr.DataArray(xr.zeros_like(output20_CO))
#D21
output21_CO = xr.DataArray(np.arange(count_T21, dtype=np.double))
output21_CO = xr.DataArray(xr.zeros_like(output21_CO))
#D22
output22_CO = xr.DataArray(np.arange(count_T22, dtype=np.double))
output22_CO = xr.DataArray(xr.zeros_like(output22_CO))
#D23
output23_CO = xr.DataArray(np.arange(count_T23, dtype=np.double))
output23_CO = xr.DataArray(xr.zeros_like(output23_CO))

In [18]:
#D0
output0_SO2 = xr.DataArray(np.arange(count_T0, dtype=np.double))
output0_SO2 = xr.DataArray(xr.zeros_like(output0_SO2))
#D1
output1_SO2 = xr.DataArray(np.arange(count_T1, dtype=np.double))
output1_SO2 = xr.DataArray(xr.zeros_like(output1_SO2))
#D2
output2_SO2 = xr.DataArray(np.arange(count_T2, dtype=np.double))
output2_SO2 = xr.DataArray(xr.zeros_like(output2_SO2))
#D3
output3_SO2 = xr.DataArray(np.arange(count_T3, dtype=np.double))
output3_SO2 = xr.DataArray(xr.zeros_like(output3_SO2))
#D4
output4_SO2 = xr.DataArray(np.arange(count_T4, dtype=np.double))
output4_SO2 = xr.DataArray(xr.zeros_like(output4_SO2))
#D5
output5_SO2 = xr.DataArray(np.arange(count_T5, dtype=np.double))
output5_SO2 = xr.DataArray(xr.zeros_like(output5_SO2))
#D6
output6_SO2 = xr.DataArray(np.arange(count_T6, dtype=np.double))
output6_SO2 = xr.DataArray(xr.zeros_like(output6_SO2))
#D7
output7_SO2 = xr.DataArray(np.arange(count_T7, dtype=np.double))
output7_SO2 = xr.DataArray(xr.zeros_like(output7_SO2))
#D8
output8_SO2 = xr.DataArray(np.arange(count_T8, dtype=np.double))
output8_SO2 = xr.DataArray(xr.zeros_like(output8_SO2))
#D9
output9_SO2 = xr.DataArray(np.arange(count_T9, dtype=np.double))
output9_SO2 = xr.DataArray(xr.zeros_like(output9_SO2))
#D10
output10_SO2 = xr.DataArray(np.arange(count_T10, dtype=np.double))
output10_SO2 = xr.DataArray(xr.zeros_like(output10_SO2))
#D11
output11_SO2 = xr.DataArray(np.arange(count_T11, dtype=np.double))
output11_SO2 = xr.DataArray(xr.zeros_like(output11_SO2))
#D12
output12_SO2 = xr.DataArray(np.arange(count_T12, dtype=np.double))
output12_SO2 = xr.DataArray(xr.zeros_like(output12_SO2))
#D13
output13_SO2 = xr.DataArray(np.arange(count_T13, dtype=np.double))
output13_SO2 = xr.DataArray(xr.zeros_like(output13_SO2))
#D14
output14_SO2 = xr.DataArray(np.arange(count_T14, dtype=np.double))
output14_SO2 = xr.DataArray(xr.zeros_like(output14_SO2))
#D15
output15_SO2 = xr.DataArray(np.arange(count_T15, dtype=np.double))
output15_SO2 = xr.DataArray(xr.zeros_like(output15_SO2))
#D16
output16_SO2 = xr.DataArray(np.arange(count_T16, dtype=np.double))
output16_SO2 = xr.DataArray(xr.zeros_like(output16_SO2))
#D17
output17_SO2 = xr.DataArray(np.arange(count_T17, dtype=np.double))
output17_SO2 = xr.DataArray(xr.zeros_like(output17_SO2))
#D18
output18_SO2 = xr.DataArray(np.arange(count_T18, dtype=np.double))
output18_SO2 = xr.DataArray(xr.zeros_like(output18_SO2))
#D19
output19_SO2 = xr.DataArray(np.arange(count_T19, dtype=np.double))
output19_SO2 = xr.DataArray(xr.zeros_like(output19_SO2))
#D20
output20_SO2 = xr.DataArray(np.arange(count_T20, dtype=np.double))
output20_SO2 = xr.DataArray(xr.zeros_like(output20_SO2))
#D21
output21_SO2 = xr.DataArray(np.arange(count_T21, dtype=np.double))
output21_SO2 = xr.DataArray(xr.zeros_like(output21_SO2))
#D22
output22_SO2 = xr.DataArray(np.arange(count_T22, dtype=np.double))
output22_SO2 = xr.DataArray(xr.zeros_like(output22_SO2))
#D23
output23_SO2 = xr.DataArray(np.arange(count_T23, dtype=np.double))
output23_SO2 = xr.DataArray(xr.zeros_like(output23_SO2))

In [19]:
# cell 16
# getting values from extracted data and putting it into an array
count_B0 = 0 
count_B1 = 0
count_B2 = 0
count_B3 = 0
count_B4 = 0
count_B5 = 0
count_B6 = 0
count_B7 = 0
count_B8 = 0
count_B9 = 0
count_B10 = 0
count_B11 = 0
count_B12 = 0
count_B13 = 0
count_B14 = 0
count_B15 = 0
count_B16 = 0
count_B17 = 0
count_B18 = 0
count_B19 = 0
count_B20 = 0
count_B21 = 0
count_B22 = 0
count_B23 = 0

col = 299
row = 459
time = 24
for i in range(col):
    for j in range(row):
        for k in range(time):
            if (np.isnan(County_CMAQ_ChemArr[0,i,j,k])==False):
                if k == 0:
                    output0_Lat[count_B0] = lat2d.values[i,j]
                    output0_Lon[count_B0] = lon2d.values[i,j]
                    output0_O3[count_B0] = O3_CV[k,i,j]
                    output0_NO[count_B0] = NO_CV[k,i,j]
                    output0_NO2[count_B0] = NO2_CV[k,i,j]
                    output0_CO[count_B0] = CO_CV[k,i,j]
                    output0_SO2[count_B0] = SO2_CV[k,i,j]
                    count_B0 = count_B0 + 1
                if k == 1:
                    output1_Lat[count_B1] = lat2d.values[i,j]
                    output1_Lon[count_B1] = lon2d.values[i,j]
                    output1_O3[count_B1] = O3_CV[k,i,j]
                    output1_NO[count_B1] = NO_CV[k,i,j]
                    output1_NO2[count_B1] = NO2_CV[k,i,j]
                    output1_CO[count_B1] = CO_CV[k,i,j]
                    output1_SO2[count_B1] = SO2_CV[k,i,j]
                    count_B1 = count_B1 + 1
                if k == 2:
                    output2_Lat[count_B2] = lat2d.values[i,j]
                    output2_Lon[count_B2] = lon2d.values[i,j]
                    output2_O3[count_B2] = O3_CV[k,i,j]
                    output2_NO[count_B2] = NO_CV[k,i,j]
                    output2_NO2[count_B2] = NO2_CV[k,i,j]
                    output2_CO[count_B2] = CO_CV[k,i,j]
                    output2_SO2[count_B2] = SO2_CV[k,i,j]
                    count_B2 = count_B2 + 1
                if k == 3:
                    output3_Lat[count_B3] = lat2d.values[i,j]
                    output3_Lon[count_B3] = lon2d.values[i,j]
                    output3_O3[count_B3] = O3_CV[k,i,j]
                    output3_NO[count_B3] = NO_CV[k,i,j]
                    output3_NO2[count_B3] = NO2_CV[k,i,j]
                    output3_CO[count_B3] = CO_CV[k,i,j]
                    output3_SO2[count_B3] = SO2_CV[k,i,j]
                    count_B3 = count_B3 + 1
                if k == 4:
                    output4_Lat[count_B4] = lat2d.values[i,j]
                    output4_Lon[count_B4] = lon2d.values[i,j]
                    output4_O3[count_B4] = O3_CV[k,i,j]
                    output4_NO[count_B4] = NO_CV[k,i,j]
                    output4_NO2[count_B4] = NO2_CV[k,i,j]
                    output4_CO[count_B4] = CO_CV[k,i,j]
                    output4_SO2[count_B4] = SO2_CV[k,i,j]
                    count_B4 = count_B4 + 1
                if k == 5:
                    output5_Lat[count_B5] = lat2d.values[i,j]
                    output5_Lon[count_B5] = lon2d.values[i,j]
                    output5_O3[count_B5] = O3_CV[k,i,j]
                    output5_NO[count_B5] = NO_CV[k,i,j]
                    output5_NO2[count_B5] = NO2_CV[k,i,j]
                    output5_CO[count_B5] = CO_CV[k,i,j]
                    output5_SO2[count_B5] = SO2_CV[k,i,j]
                    count_B5 = count_B5 + 1
                if k == 6:
                    output6_Lat[count_B6] = lat2d.values[i,j]
                    output6_Lon[count_B6] = lon2d.values[i,j]
                    output6_O3[count_B6] = O3_CV[k,i,j]
                    output6_NO[count_B6] = NO_CV[k,i,j]
                    output6_NO2[count_B6] = NO2_CV[k,i,j]
                    output6_CO[count_B6] = CO_CV[k,i,j]
                    output6_SO2[count_B6] = SO2_CV[k,i,j]
                    count_B6 = count_B6 + 1
                if k == 7:
                    output7_Lat[count_B7] = lat2d.values[i,j]
                    output7_Lon[count_B7] = lon2d.values[i,j]
                    output7_O3[count_B7] = O3_CV[k,i,j]
                    output7_NO[count_B7] = NO_CV[k,i,j]
                    output7_NO2[count_B7] = NO2_CV[k,i,j]
                    output7_CO[count_B7] = CO_CV[k,i,j]
                    output7_SO2[count_B7] = SO2_CV[k,i,j]
                    count_B7 = count_B7 + 1
                if k == 8:
                    output8_Lat[count_B8] = lat2d.values[i,j]
                    output8_Lon[count_B8] = lon2d.values[i,j]
                    output8_O3[count_B8] = O3_CV[k,i,j]
                    output8_NO[count_B8] = NO_CV[k,i,j]
                    output8_NO2[count_B8] = NO2_CV[k,i,j]
                    output8_CO[count_B8] = CO_CV[k,i,j]
                    output8_SO2[count_B8] = SO2_CV[k,i,j]
                    count_B8 = count_B8 + 1
                if k == 9:
                    output9_Lat[count_B9] = lat2d.values[i,j]
                    output9_Lon[count_B9] = lon2d.values[i,j]
                    output9_O3[count_B9] = O3_CV[k,i,j]
                    output9_NO[count_B9] = NO_CV[k,i,j]
                    output9_NO2[count_B9] = NO2_CV[k,i,j]
                    output9_CO[count_B9] = CO_CV[k,i,j]
                    output9_SO2[count_B9] = SO2_CV[k,i,j]
                    count_B9 = count_B9 + 1
                if k == 10:
                    output10_Lat[count_B10] = lat2d.values[i,j]
                    output10_Lon[count_B10] = lon2d.values[i,j]
                    output10_O3[count_B10] = O3_CV[k,i,j]
                    output10_NO[count_B10] = NO_CV[k,i,j]
                    output10_NO2[count_B10] = NO2_CV[k,i,j]
                    output10_CO[count_B10] = CO_CV[k,i,j]
                    output10_SO2[count_B10] = SO2_CV[k,i,j]
                    count_B10 = count_B10 + 1
                if k == 11:
                    output11_Lat[count_B11] = lat2d.values[i,j]
                    output11_Lon[count_B11] = lon2d.values[i,j]
                    output11_O3[count_B11] = O3_CV[k,i,j]
                    output11_NO[count_B11] = NO_CV[k,i,j]
                    output11_NO2[count_B11] = NO2_CV[k,i,j]
                    output11_CO[count_B11] = CO_CV[k,i,j]
                    output11_SO2[count_B11] = SO2_CV[k,i,j]
                    count_B11 = count_B11 + 1
                if k == 12:
                    output12_Lat[count_B12] = lat2d.values[i,j]
                    output12_Lon[count_B12] = lon2d.values[i,j]
                    output12_O3[count_B12] = O3_CV[k,i,j]
                    output12_NO[count_B12] = NO_CV[k,i,j]
                    output12_NO2[count_B12] = NO2_CV[k,i,j]
                    output12_CO[count_B12] = CO_CV[k,i,j]
                    output12_SO2[count_B12] = SO2_CV[k,i,j]
                    count_B12 = count_B12 + 1
                if k == 13:
                    output13_Lat[count_B13] = lat2d.values[i,j]
                    output13_Lon[count_B13] = lon2d.values[i,j]
                    output13_O3[count_B13] = O3_CV[k,i,j]
                    output13_NO[count_B13] = NO_CV[k,i,j]
                    output13_NO2[count_B13] = NO2_CV[k,i,j]
                    output13_CO[count_B13] = CO_CV[k,i,j]
                    output13_SO2[count_B13] = SO2_CV[k,i,j]
                    count_B13 = count_B13 + 1
                if k == 14:
                    output14_Lat[count_B14] = lat2d.values[i,j]
                    output14_Lon[count_B14] = lon2d.values[i,j]
                    output14_O3[count_B14] = O3_CV[k,i,j]
                    output14_NO[count_B14] = NO_CV[k,i,j]
                    output14_NO2[count_B14] = NO2_CV[k,i,j]
                    output14_CO[count_B14] = CO_CV[k,i,j]
                    output14_SO2[count_B14] = SO2_CV[k,i,j]
                    count_B14 = count_B14 + 1
                if k == 15:
                    output15_Lat[count_B15] = lat2d.values[i,j]
                    output15_Lon[count_B15] = lon2d.values[i,j]
                    output15_O3[count_B15] = O3_CV[k,i,j]
                    output15_NO[count_B15] = NO_CV[k,i,j]
                    output15_NO2[count_B15] = NO2_CV[k,i,j]
                    output15_CO[count_B15] = CO_CV[k,i,j]
                    output15_SO2[count_B15] = SO2_CV[k,i,j]
                    count_B15 = count_B15 + 1
                if k == 16:
                    output16_Lat[count_B16] = lat2d.values[i,j]
                    output16_Lon[count_B16] = lon2d.values[i,j]
                    output16_O3[count_B16] = O3_CV[k,i,j]
                    output16_NO[count_B16] = NO_CV[k,i,j]
                    output16_NO2[count_B16] = NO2_CV[k,i,j]
                    output16_CO[count_B16] = CO_CV[k,i,j]
                    output16_SO2[count_B16] = SO2_CV[k,i,j]
                    count_B16 = count_B16 + 1
                if k == 17:
                    output17_Lat[count_B17] = lat2d.values[i,j]
                    output17_Lon[count_B17] = lon2d.values[i,j]
                    output17_O3[count_B17] = O3_CV[k,i,j]
                    output17_NO[count_B17] = NO_CV[k,i,j]
                    output17_NO2[count_B17] = NO2_CV[k,i,j]
                    output17_CO[count_B17] = CO_CV[k,i,j]
                    output17_SO2[count_B17] = SO2_CV[k,i,j]
                    count_B17 = count_B17 + 1
                if k == 18:
                    output18_Lat[count_B18] = lat2d.values[i,j]
                    output18_Lon[count_B18] = lon2d.values[i,j]
                    output18_O3[count_B18] = O3_CV[k,i,j]
                    output18_NO[count_B18] = NO_CV[k,i,j]
                    output18_NO2[count_B18] = NO2_CV[k,i,j]
                    output18_CO[count_B18] = CO_CV[k,i,j]
                    output18_SO2[count_B18] = SO2_CV[k,i,j]
                    count_B18 = count_B18 + 1
                if k == 19:
                    output19_Lat[count_B19] = lat2d.values[i,j]
                    output19_Lon[count_B19] = lon2d.values[i,j]
                    output19_O3[count_B19] = O3_CV[k,i,j]
                    output19_NO[count_B19] = NO_CV[k,i,j]
                    output19_NO2[count_B19] = NO2_CV[k,i,j]
                    output19_CO[count_B19] = CO_CV[k,i,j]
                    output19_SO2[count_B19] = SO2_CV[k,i,j]
                    count_B19 = count_B19 + 1
                if k == 20:
                    output20_Lat[count_B20] = lat2d.values[i,j]
                    output20_Lon[count_B20] = lon2d.values[i,j]
                    output20_O3[count_B20] = O3_CV[k,i,j]
                    output20_NO[count_B20] = NO_CV[k,i,j]
                    output20_NO2[count_B20] = NO2_CV[k,i,j]
                    output20_CO[count_B20] = CO_CV[k,i,j]
                    output20_SO2[count_B20] = SO2_CV[k,i,j]
                    count_B20 = count_B20 + 1
                if k == 21:
                    output21_Lat[count_B21] = lat2d.values[i,j]
                    output21_Lon[count_B21] = lon2d.values[i,j]
                    output21_O3[count_B21] = O3_CV[k,i,j]
                    output21_NO[count_B21] = NO_CV[k,i,j]
                    output21_NO2[count_B21] = NO2_CV[k,i,j]
                    output21_CO[count_B21] = CO_CV[k,i,j]
                    output21_SO2[count_B21] = SO2_CV[k,i,j]
                    count_B21 = count_B21 + 1
                if k == 22:
                    output22_Lat[count_B22] = lat2d.values[i,j]
                    output22_Lon[count_B22] = lon2d.values[i,j]
                    output22_O3[count_B22] = O3_CV[k,i,j]
                    output22_NO[count_B22] = NO_CV[k,i,j]
                    output22_NO2[count_B22] = NO2_CV[k,i,j]
                    output22_CO[count_B22] = CO_CV[k,i,j]
                    output22_SO2[count_B22] = SO2_CV[k,i,j]
                    count_B22 = count_B22 + 1
                if k == 23:
                    output23_Lat[count_B23] = lat2d.values[i,j]
                    output23_Lon[count_B23] = lon2d.values[i,j]
                    output23_O3[count_B23] = O3_CV[k,i,j]
                    output23_NO[count_B23] = NO_CV[k,i,j]
                    output23_NO2[count_B23] = NO2_CV[k,i,j]
                    output23_CO[count_B23] = CO_CV[k,i,j]
                    output23_SO2[count_B23] = SO2_CV[k,i,j]
                    count_B23 = count_B23 + 1

In [20]:
# cell 17
# checking output
print(output1_O3)
print(output10_NO2)
#print(type(output0_O3)

<xarray.DataArray (dim_0: 30)>
array([0.01958811, 0.0193124 , 0.01909408, 0.01813515, 0.01764398,
       0.01667057, 0.0196434 , 0.01891853, 0.01865944, 0.01769139,
       0.01718273, 0.0195251 , 0.01914203, 0.01802992, 0.01788807,
       0.01721268, 0.01993028, 0.01855187, 0.01899183, 0.01763581,
       0.01793334, 0.0193387 , 0.0184829 , 0.01861734, 0.01782973,
       0.01763088, 0.01877584, 0.01760598, 0.01806608, 0.01717748])
Dimensions without coordinates: dim_0
<xarray.DataArray (dim_0: 30)>
array([0.00033978, 0.00068928, 0.00030206, 0.00031466, 0.0003761 ,
       0.00054386, 0.00061312, 0.0005005 , 0.00035786, 0.0002533 ,
       0.00026673, 0.00058813, 0.00069932, 0.0008182 , 0.00036395,
       0.00023359, 0.0016644 , 0.00063942, 0.00066109, 0.00070411,
       0.00028077, 0.00055584, 0.00052962, 0.00045989, 0.00037583,
       0.00022821, 0.0009494 , 0.00082461, 0.00072554, 0.00032384])
Dimensions without coordinates: dim_0


In [21]:
# cell 18
# creating new dataset with new arrays
CMAQ_Extract_O3 = xr.Dataset(
    data_vars=dict(
        Lat = (["x"], output0_Lat.data),
        Lon = (["x"], output0_Lon.data),
        O3_T0 = (["x"], output0_O3.data),
        O3_T1 = (["x"], output1_O3.data),
        O3_T2 = (["x"], output2_O3.data),
        O3_T3 = (["x"], output3_O3.data),
        O3_T4 = (["x"], output4_O3.data),
        O3_T5 = (["x"], output5_O3.data),
        O3_T6 = (["x"], output6_O3.data),
        O3_T7 = (["x"], output7_O3.data),
        O3_T8 = (["x"], output8_O3.data),
        O3_T9 = (["x"], output9_O3.data),
        O3_T10 = (["x"], output10_O3.data),
        O3_T11 = (["x"], output11_O3.data),
        O3_T12 = (["x"], output12_O3.data),
        O3_T13 = (["x"], output13_O3.data),
        O3_T14 = (["x"], output14_O3.data),
        O3_T15 = (["x"], output15_O3.data),
        O3_T16 = (["x"], output16_O3.data),
        O3_T17 = (["x"], output17_O3.data),
        O3_T18 = (["x"], output18_O3.data),
        O3_T19 = (["x"], output19_O3.data),
        O3_T20 = (["x"], output20_O3.data),
        O3_T21 = (["x"], output21_O3.data),
        O3_T22 = (["x"], output22_O3.data),
        O3_T23 = (["x"], output23_O3.data),
    ),
    attrs=dict(
        description="Ozone concentrations from every hour in a 24 hour period",
        units="Parts per Million (ppm)",
        Latitude= " Extracted from 34.297882N to 34.95478058N ",
        Longitude= " Extracted from -79.46142578W to -78.80546570W "  
    ),
)
CMAQ_Extract_O3     

CMAQ_Extract_NO = xr.Dataset(
    data_vars=dict(
        Lat = (["x"], output0_Lat.data),
        Lon = (["x"], output0_Lon.data),
        NO_T0 = (["x"], output0_NO.data),
        NO_T1 = (["x"], output1_NO.data),
        NO_T2 = (["x"], output2_NO.data),
        NO_T3 = (["x"], output3_NO.data),
        NO_T4 = (["x"], output4_NO.data),
        NO_T5 = (["x"], output5_NO.data),
        NO_T6 = (["x"], output6_NO.data),
        NO_T7 = (["x"], output7_NO.data),
        NO_T8 = (["x"], output8_NO.data),
        NO_T9 = (["x"], output9_NO.data),
        NO_T10 = (["x"], output10_NO.data),
        NO_T11 = (["x"], output11_NO.data),
        NO_T12 = (["x"], output12_NO.data),
        NO_T13 = (["x"], output13_NO.data),
        NO_T14 = (["x"], output14_NO.data),
        NO_T15 = (["x"], output15_NO.data),
        NO_T16 = (["x"], output16_NO.data),
        NO_T17 = (["x"], output17_NO.data),
        NO_T18 = (["x"], output18_NO.data),
        NO_T19 = (["x"], output19_NO.data),
        NO_T20 = (["x"], output20_NO.data),
        NO_T21 = (["x"], output21_NO.data),
        NO_T22 = (["x"], output22_NO.data),
        NO_T23 = (["x"], output23_NO.data),
    ),
    attrs=dict(
        description="Nitrogen monoxide concentrations from every hour in a 24 hour period",
        units="Parts per Million (ppm)",
        Latitude= " Extracted from 34.297882N to 34.95478058N ",
        Longitude= " Extracted from -79.46142578W to -78.80546570W "  
    ),
)
CMAQ_Extract_NO

<xarray.Dataset>
Dimensions:  (x: 30)
Dimensions without coordinates: x
Data variables: (12/26)
    Lat      (x) float64 34.31 34.42 34.4 34.38 ... 34.95 34.93 34.91 34.89
    Lon      (x) float64 -79.41 -79.38 -79.25 -79.13 ... -79.13 -79.0 -78.87
    NO_T0    (x) float64 6.709e-10 3.451e-07 3.319e-11 ... 3.748e-09 1.404e-09
    NO_T1    (x) float64 2.519e-09 7.38e-07 6.092e-11 ... 1.023e-08 1.282e-08
    NO_T2    (x) float64 9.694e-09 1.617e-06 1.677e-10 ... 5.507e-08 8.412e-08
    NO_T3    (x) float64 4.026e-08 3.55e-06 4.687e-10 ... 4.147e-07 6.068e-07
    ...       ...
    NO_T18   (x) float64 0.0001035 0.000292 7.439e-05 ... 0.0001624 7.353e-05
    NO_T19   (x) float64 8.618e-05 0.0001787 8.795e-05 ... 0.0001431 7.617e-05
    NO_T20   (x) float64 5.627e-07 3.253e-06 1.064e-07 ... 8.202e-07 4.205e-07
    NO_T21   (x) float64 2.969e-07 3.141e-06 1.838e-08 ... 5.759e-07 1.958e-07
    NO_T22   (x) float64 1.634e-07 3.057e-06 8.082e-09 ... 3.272e-07 1.136e-07
    NO_T23   (x) float64 8.086e-08 3.625e-06 2.668e-09 ... 2.497e-07 1.114e-07
Attributes:
    description:  Nitrogen monoxide concentrations from every hour in a 24 ho...
    units:        Parts per Million (ppm)
    Latitude:      Extracted from 34.297882N to 34.95478058N 
    Longitude:     Extracted from -79.46142578W to -78.80546570W

In [22]:
CMAQ_Extract_NO2 = xr.Dataset(
  data_vars=dict(
    Lat = (["x"], output0_Lat.data),
    Lon = (["x"], output0_Lon.data),
    NO2_T0 = (["x"], output0_NO2.data),
    NO2_T1 = (["x"], output1_NO2.data),
    NO2_T2 = (["x"], output2_NO2.data),
    NO2_T3 = (["x"], output3_NO2.data),
    NO2_T4 = (["x"], output4_NO2.data),
    NO2_T5 = (["x"], output5_NO2.data),
    NO2_T6 = (["x"], output6_NO2.data),
    NO2_T7 = (["x"], output7_NO2.data),
    NO2_T8 = (["x"], output8_NO2.data),
    NO2_T9 = (["x"], output9_NO2.data),
    NO2_T10 = (["x"], output10_NO2.data),
    NO2_T11 = (["x"], output11_NO2.data),
    NO2_T12 = (["x"], output12_NO2.data),
    NO2_T13 = (["x"], output13_NO2.data),
    NO2_T14 = (["x"], output14_NO2.data),
    NO2_T15 = (["x"], output15_NO2.data),
    NO2_T16 = (["x"], output16_NO2.data),
    NO2_T17 = (["x"], output17_NO2.data),
    NO2_T18 = (["x"], output18_NO2.data),
    NO2_T19 = (["x"], output19_NO2.data),
    NO2_T20 = (["x"], output20_NO2.data),
    NO2_T21 = (["x"], output21_NO2.data),
    NO2_T22 = (["x"], output22_NO2.data),
    NO2_T23 = (["x"], output23_NO2.data),
  ),
  attrs=dict(
    description="Nitrogen dioxide concentrations from every hour in a 24 hour period",
    units="Parts per Million (ppm)",
    Latitude= " Extracted from 34.297882N to 34.95478058N ",
    Longitude= " Extracted from -79.46142578W to -78.80546570W "  
  ),
)
CMAQ_Extract_NO2

CMAQ_Extract_CO = xr.Dataset(
  data_vars=dict(
    Lat = (["x"], output0_Lat.data),
    Lon = (["x"], output0_Lon.data),
    CO_T0 = (["x"], output0_CO.data),
    CO_T1 = (["x"], output1_CO.data),
    CO_T2 = (["x"], output2_CO.data),
    CO_T3 = (["x"], output3_CO.data),
    CO_T4 = (["x"], output4_CO.data),
    CO_T5 = (["x"], output5_CO.data),
    CO_T6 = (["x"], output6_CO.data),
    CO_T7 = (["x"], output7_CO.data),
    CO_T8 = (["x"], output8_CO.data),
    CO_T9 = (["x"], output9_CO.data),
    CO_T10 = (["x"], output10_CO.data),
    CO_T11 = (["x"], output11_CO.data),
    CO_T12 = (["x"], output12_CO.data),
    CO_T13 = (["x"], output13_CO.data),
    CO_T14 = (["x"], output14_CO.data),
    CO_T15 = (["x"], output15_CO.data),
    CO_T16 = (["x"], output16_CO.data),
    CO_T17 = (["x"], output17_CO.data),
    CO_T18 = (["x"], output18_CO.data),
    CO_T19 = (["x"], output19_CO.data),
    CO_T20 = (["x"], output20_CO.data),
    CO_T21 = (["x"], output21_CO.data),
    CO_T22 = (["x"], output22_CO.data),
    CO_T23 = (["x"], output23_CO.data),
  ),
  attrs=dict(
    description="Carbon monoxide concentrations from every hour in a 24 hour period",
    units="Parts per Million (ppm)",
    Latitude= " Extracted from 34.297882N to 34.95478058N ",
    Longitude= " Extracted from -79.46142578W to -78.80546570W "  
  ),
)
CMAQ_Extract_CO

CMAQ_Extract_SO2 = xr.Dataset(
  data_vars=dict(
    Lat = (["x"], output0_Lat.data),
    Lon = (["x"], output0_Lon.data),
    SO2_T0 = (["x"], output0_SO2.data),
    SO2_T1 = (["x"], output1_SO2.data),
    SO2_T2 = (["x"], output2_SO2.data),
    SO2_T3 = (["x"], output3_SO2.data),
    SO2_T4 = (["x"], output4_SO2.data),
    SO2_T5 = (["x"], output5_SO2.data),
    SO2_T6 = (["x"], output6_SO2.data),
    SO2_T7 = (["x"], output7_SO2.data),
    SO2_T8 = (["x"], output8_SO2.data),
    SO2_T9 = (["x"], output9_SO2.data),
    SO2_T10 = (["x"], output10_SO2.data),
    SO2_T11 = (["x"], output11_SO2.data),
    SO2_T12 = (["x"], output12_SO2.data),
    SO2_T13 = (["x"], output13_SO2.data),
    SO2_T14 = (["x"], output14_SO2.data),
    SO2_T15 = (["x"], output15_SO2.data),
    SO2_T16 = (["x"], output16_SO2.data),
    SO2_T17 = (["x"], output17_SO2.data),
    SO2_T18 = (["x"], output18_SO2.data),
    SO2_T19 = (["x"], output19_SO2.data),
    SO2_T20 = (["x"], output20_SO2.data),
    SO2_T21 = (["x"], output21_SO2.data),
    SO2_T22 = (["x"], output22_SO2.data),
    SO2_T23 = (["x"], output23_SO2.data),
  ),
  attrs=dict(
    description="SO2 concentrations from every hour in a 24 hour period",
    units="Parts per Million (ppm)",
    Latitude= " Extracted from 34.297882N to 34.95478058N ",
    Longitude= " Extracted from -79.46142578W to -78.80546570W "  
  ),
)
CMAQ_Extract_SO2

<xarray.Dataset>
Dimensions:  (x: 30)
Dimensions without coordinates: x
Data variables: (12/26)
    Lat      (x) float64 34.31 34.42 34.4 34.38 ... 34.95 34.93 34.91 34.89
    Lon      (x) float64 -79.41 -79.38 -79.25 -79.13 ... -79.13 -79.0 -78.87
    SO2_T0   (x) float64 0.0009611 0.001008 0.0008218 ... 6.121e-05 7.803e-05
    SO2_T1   (x) float64 0.0006842 0.0007573 0.0005994 ... 8.893e-05 0.000104
    SO2_T2   (x) float64 0.0004526 0.0005201 0.0003891 ... 0.0001246 0.0001326
    SO2_T3   (x) float64 0.000295 0.0003043 0.0002405 ... 0.0001713 0.000145
    ...       ...
    SO2_T18  (x) float64 0.0002024 0.0006508 0.0008827 ... 5.072e-05 4.829e-05
    SO2_T19  (x) float64 0.0002184 0.0005712 0.0002447 ... 5.77e-05 5.288e-05
    SO2_T20  (x) float64 7.47e-05 6.617e-05 3.827e-05 ... 3.827e-05 3.721e-05
    SO2_T21  (x) float64 8.955e-05 6.953e-05 4.126e-05 ... 3.948e-05 4.1e-05
    SO2_T22  (x) float64 0.0001456 9.372e-05 4.879e-05 ... 4.092e-05 4.368e-05
    SO2_T23  (x) float64 0.0002593 0.0001911 8.381e-05 ... 4.133e-05 4.268e-05
Attributes:
    description:  SO2 concentrations from every hour in a 24 hour period
    units:        Parts per Million (ppm)
    Latitude:      Extracted from 34.297882N to 34.95478058N 
    Longitude:     Extracted from -79.46142578W to -78.80546570W

In [23]:
# cell 19
# saving extracted data
# CMAQ_Extract_O3.to_netcdf("O3_data_24hr_20130xxx.nc")
# CMAQ_Extract_NO.to_netcdf("NO_data_24hr_20130xxx.nc")
# CMAQ_Extract_NO2.to_netcdf("NO2_data_24hr_20130xxx.nc")
# CMAQ_Extract_CO.to_netcdf("CO_data_24hr_20130xxx.nc")
# CMAQ_Extract_SO2.to_netcdf("SO2_data_24hr_20130xxx.nc")

In [24]:
CMAQ_O3 = CMAQ_Extract_O3.to_dataframe()
CMAQ_NO = CMAQ_Extract_NO.to_dataframe()
CMAQ_NO2 = CMAQ_Extract_NO2.to_dataframe()
CMAQ_CO = CMAQ_Extract_CO.to_dataframe()
CMAQ_SO2 = CMAQ_Extract_SO2.to_dataframe()

In [25]:
CMAQ_O3.to_csv("Output_CSVs/20130715/O3_data_24hr_20130715and0531.csv")
CMAQ_NO.to_csv("Output_CSVs/20130715/NO_data_24hr_20130715and0531.csv")
CMAQ_NO2.to_csv("Output_CSVs/20130715/NO2_data_24hr_20130715and0531.csv")
CMAQ_CO.to_csv("Output_CSVs/20130715/CO_data_24hr_20130715and0531.csv")
CMAQ_SO2.to_csv("Output_CSVs/20130715/SO2_data_24hr_20130715and0531.csv")

In [ ]:
# cell 20
# closing files

ACON_file.close()
ACON_file_MCIP.close()
ACON_file_2.close()
ACON_file_MCIP_2.close()